In [54]:
import numpy as np
import sklearn.utils as skutils
from sklearn.utils.extmath import svd_flip
from scipy.sparse import csc_matrix, csr_matrix
from scipy.sparse.linalg import LinearOperator, svds


In [ ]:
adata = sc.read_10x_mtx(
    'data/hg19/',  # the directory with the `.mtx` file
    var_names='gene_symbols',                # use gene symbols for the variable names (variables-axis index)
    cache=True)

### How to perform PCA on sparse matrix

#### When zero_center is True

In [3]:
def _pca_with_sparse(X, comp_num, solver='arpark', mu=None, random_state=None):
    random_state = skutils.check_random_state(random_state)

In [12]:
help(skutils.check_random_state)

Help on function check_random_state in module sklearn.utils.validation:

check_random_state(seed)
    Turn seed into a np.random.RandomState instance
    
    Parameters
    ----------
    seed : None | int | instance of RandomState
        If seed is None, return the RandomState singleton used by np.random.
        If seed is an int, return a new RandomState instance seeded with seed.
        If seed is already a RandomState instance, return it.
        Otherwise raise ValueError.



In [29]:
row  = np.array([0, 3, 1, 0, 2])
col  = np.array([0, 3, 1, 2, 5])
data = np.array([4, 5, 7, 9, 2])
X = csc_matrix((data, (row, col)), shape=(4, 6))

In [27]:
random_state = 1
random_state = skutils.check_random_state(random_state)
np.random.set_state(random_state.get_state())
random_init = np.random.rand(np.min(X.shape))
X = check_array(X, accept_sparse=['csr', 'csc'])

In [42]:
X.mean(axis=0)

matrix([[1.  , 1.75, 2.25, 1.25, 0.  , 0.5 ]])

In [43]:
X.mean(axis=0).A

array([[1.  , 1.75, 2.25, 1.25, 0.  , 0.5 ]])

In [44]:
X.mean(axis=0).A.flatten()

array([1.  , 1.75, 2.25, 1.25, 0.  , 0.5 ])

In [45]:
X.mean(axis=0).A.flatten()[None, :]

array([[1.  , 1.75, 2.25, 1.25, 0.  , 0.5 ]])

In [46]:
mu = X.mean(axis=0).A.flatten()[None, :]

In [47]:
mdot = mmat = mu.dot
mhdot = mhmat = mu.T.dot
Xdot = Xmat = X.dot
XHdot = XHdot = X.T.conj().dot
ones = np.ones(X.shape[0])[None, :].dot

In [57]:
matvec = lambda x: X @ x.T - mu @ X.T
matmat = lambda x: X @ x.T - mu @ X.T
rmatvec = lambda x: X.T.conj() @ x.T - mu.T @ (np.ones(X.shape[0])[None, :] @ x.T).T
rmatmat = lambda x: X.T.conj() @ x.T - mu.T @ (np.ones(X.shape[0])[None, :] @ x.T).T

In [58]:
XL = LinearOperator(
        matvec=matvec,
        dtype=X.dtype,
        matmat=matmat,
        shape=X.shape,
        rmatvec=rmatvec,
        rmatmat=rmatmat,
    )

In [62]:
help(LinearOperator)

Help on class LinearOperator in module scipy.sparse.linalg.interface:

class LinearOperator(builtins.object)
 |  LinearOperator(*args, **kwargs)
 |  
 |  Common interface for performing matrix vector products
 |  
 |  Many iterative methods (e.g. cg, gmres) do not need to know the
 |  individual entries of a matrix to solve a linear system A*x=b.
 |  Such solvers only require the computation of matrix vector
 |  products, A*v where v is a dense vector.  This class serves as
 |  an abstract interface between iterative solvers and matrix-like
 |  objects.
 |  
 |  To construct a concrete LinearOperator, either pass appropriate
 |  callables to the constructor of this class, or subclass it.
 |  
 |  A subclass must implement either one of the methods ``_matvec``
 |  and ``_matmat``, and the attributes/properties ``shape`` (pair of
 |  integers) and ``dtype`` (may be None). It may call the ``__init__``
 |  on this class to have these attributes validated. Implementing
 |  ``_matvec`` autom

In [61]:
help(svd_flip)

Help on function svd_flip in module sklearn.utils.extmath:

svd_flip(u, v, u_based_decision=True)
    Sign correction to ensure deterministic output from SVD.
    
    Adjusts the columns of u and the rows of v such that the loadings in the
    columns in u that are largest in absolute value are always positive.
    
    Parameters
    ----------
    u : ndarray
        u and v are the output of `linalg.svd` or
        :func:`~sklearn.utils.extmath.randomized_svd`, with matching inner
        dimensions so one can compute `np.dot(u * s, v)`.
    
    v : ndarray
        u and v are the output of `linalg.svd` or
        :func:`~sklearn.utils.extmath.randomized_svd`, with matching inner
        dimensions so one can compute `np.dot(u * s, v)`.
    
    u_based_decision : boolean, (default=True)
        If True, use the columns of u as the basis for sign flipping.
        Otherwise, use the rows of v. The choice of which variable to base the
        decision on is generally algorithm depe

In [52]:
np.arange(10).reshape(2,5).dot((np.arange(2) - 2))

ValueError: shapes (2,5) and (2,) not aligned: 5 (dim 1) != 2 (dim 0)

In [55]:
X

<4x6 sparse matrix of type '<class 'numpy.int64'>'
	with 5 stored elements in Compressed Sparse Column format>

In [56]:
X.T

<6x4 sparse matrix of type '<class 'numpy.int64'>'
	with 5 stored elements in Compressed Sparse Row format>